## deltaLens Data Compare Sample

In [1]:
#imports

from delta_lens.deltaLens import DeltaLens, EntityComparer
from delta_lens.config import Config, Entity, Side, Transform, Defaults,ReferenceDataset, load_config
from delta_lens.sqliteExport import *
from datetime import date

In [2]:
#config

config = load_config("data/compare.config.json")


In [3]:
runName = f'tradeCompare_{date.today().strftime("%Y-%m-%d")}'

comparator = DeltaLens(runName,config)

comparator.execute(continue_on_error=True)



#### entity_compare_results
Table containing summary of each entity comparison

In [4]:
comparator.con.execute("SELECT * FROM  entity_compare_results").fetch_df()

,entity,rows_left,rows_right,rows_fully_matched,error_text,success
0,trade,42991209,43000000,42982369,None,1


#### [entitiy]_compare_field_summary
Entity specific table containing summary of field comparison

In [5]:
comparator.con.execute("SELECT * FROM trade_compare_field_summary").fetch_df()

,field,total,matches,match_percentage
0,price,42991209,42982369.0,99.980003
1,quantity,42991209,42991209.0,100.000000
2,symbol,42991209,42991209.0,100.000000
3,timestamp,42991209,42991209.0,100.000000
4,trade_type,42991209,42991209.0,100.000000


#### [entity]_compare
Entity specific table containing details of comparison

In [6]:
comparator.con.execute(" WITH CTE as (SELECT * FROM  trade_compare WHERE _full_match = 0) SELECT * FROM CTE USING SAMPLE 5;").fetch_df()

,trade_id,price_left,price_right,price_match,quantity_left,quantity_right,quantity_match,symbol_left,symbol_right,symbol_match,timestamp_left,timestamp_right,timestamp_match,trade_type_left,trade_type_right,trade_type_match,_exists_left,_exists_right,_full_match
0,5888662,844.26,495.06,False,4748,4748,True,AAPL,AAPL,True,2024-09-24 04:57:51,2024-09-24 04:57:51,True,BUY,BUY,True,True,True,False
1,22936090,451.67,493.54,False,7513,7513,True,WMT,WMT,True,2024-08-12 05:00:57,2024-08-12 05:00:57,True,BUY,BUY,True,True,True,False
2,18041008,NaN,611.21,False,<NA>,7930,False,None,NVDA,False,NaT,2024-10-15 05:00:05,False,None,SELL,False,False,True,False
3,22680416,NaN,482.96,False,<NA>,1603,False,None,TSLA,False,NaT,2024-06-29 05:00:55,False,None,BUY,False,False,True,False
4,14311087,NaN,348.40,False,<NA>,6026,False,None,BAC,False,NaT,2024-08-16 04:59:24,False,None,BUY,False,False,True,False


#### Export to sqlite
Function to export above tables to on-disk format. Sqlite is a popular single file database, well supported in many languages,  for which lots of query tools exist. See [Datasette](https://datasette.io/) package.

In [8]:
export_to_sqlite(comparator.con, f'{runName}.sqlite')

'tradeCompare_2025-02-14.sqlite'